In [3]:
!nvidia-smi

Sat Sep 17 04:07:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import torch
print(torch.cuda.is_available())

device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
print(device)
if device==True:
    print(torch.cuda.current_device())
    print(torch.cuda.device_count())

True
cuda


In [6]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [7]:
import os
os.getcwd()

'/content'

In [8]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import random

from tqdm.auto import tqdm, trange
import copy
import glob
import pickle

import numpy as np
from numpy import argmax
import pandas as pd
import seaborn as sns
import PIL.Image as Image
import matplotlib.pyplot as plt
import scipy.ndimage

import nibabel as nib

from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_auc_score, \
                            mean_squared_error, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
# from sklearn.inspection import permutation_importance
from sklearn.datasets import load_boston
from sklearn.svm import SVC
from xgboost import XGBClassifier
import lightgbm
from lightgbm import LGBMClassifier
from lightgbm import plot_importance
# import joblib

# import shap

from IPython.display import display as disp
import multiprocessing as mp


In [9]:
from zipfile import ZipFile
from tqdm.auto import tqdm
import os

with ZipFile('/content/drive/MyDrive/study/ASTROSIGHT/dataset/dicom_to_np.zip') as zipper:
# zipper.extractall(path='/content/datas')
    for file in tqdm(iterable=zipper.namelist(), total=len(zipper.namelist())):
        zipper.extract(member=file, path='/')

  0%|          | 0/651 [00:00<?, ?it/s]

In [10]:
from zipfile import ZipFile
from tqdm.auto import tqdm
import os

with ZipFile('/content/drive/MyDrive/study/ASTROSIGHT/model_save/only_image_external_results.zip') as zipper:
# zipper.extractall(path='/content/datas')
    for file in tqdm(iterable=zipper.namelist(), total=len(zipper.namelist())):
        zipper.extract(member=file, path='/content/only_image_external_results/')

  0%|          | 0/2235 [00:00<?, ?it/s]

In [11]:
FILE_EXTENSION = ['.png', '.PNG', '.jpg', '.JPG', '.dcm', '.DCM', '.raw', '.RAW', '.svs', '.SVS']
IMG_EXTENSION = ['.png', '.PNG', '.jpg', '.JPG', '.jpeg', '.JPEG']
DCM_EXTENSION = ['.dcm', '.DCM']
RAW_EXTENSION = ['.raw', '.RAW']
NIFTI_EXTENSION = ['.nii']
NP_EXTENSION = ['.npy']

import os
import numpy as np
import pandas as pd
from PIL import Image
import nibabel as nib
import scipy.ndimage
import matplotlib.pyplot as plt


def check_extension(filename, extension_ls=FILE_EXTENSION):
    return any(filename.endswith(extension) for extension in extension_ls)


def load_file_path(folder_path, extension_ls=FILE_EXTENSION, all_sub_folders=False):
    """find 'IMG_EXTENSION' file paths in folder.
    
    Parameters:
        folder_path (str) -- folder directory
        extension_ls (list) -- list of extensions
        all_sub_folders(bool) -- check all sub directories
    
    Return:
        file_paths (list) -- list of 'extension_ls' file paths
    """
    
    file_paths = []
    assert os.path.isdir(folder_path), f'{folder_path} is not a valid directory'

    for root, _, fnames in sorted(os.walk(folder_path)):
        for fname in fnames:
            if check_extension(fname, extension_ls):
                path = os.path.join(root, fname)
                file_paths.append(path)
        if not all_sub_folders:
            break

    return file_paths[:]


def gen_new_dir(new_dir):
    try: 
        if not os.path.exists(new_dir): 
            os.makedirs(new_dir) 
            #print(f"New directory!: {new_dir}")
    except OSError: 
        print("Error: Failed to create the directory.")
        

def normalize(img_3d):
    img_3d = img_3d - img_3d.min()
    if img_3d.max() != 0:
        img_3d = img_3d / img_3d.max()
    #img_3d = (img_3d-img_3d.min()) / (img_3d.max()-img_3d.min())
    return img_3d.astype(np.float32)


def z_normalize(img_3d):
    return (img_3d - 0.5) / .5


def read_nifti_file(file_path):
    scan = nib.load(file_path)
    scan = scan.get_fdata()
    return scan.transpose(2,1,0)


def resample_3d(image_3d, dsize=(36,256,256)):
    rounded_resize_factor = np.array(dsize) / image_3d.shape
    
    return scipy.ndimage.interpolation.zoom(image_3d, rounded_resize_factor, mode='nearest')


def sample_stack(stack, rows=6, cols=6, start_with=0, show_every=1, vmin=0, vmax=255):
    fig, ax = plt.subplots(rows,cols,figsize=[18,20])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/cols), int(i%cols)].set_title(f'slice {ind}')
        ax[int(i/cols), int(i%cols)].imshow(stack[ind])
        ax[int(i/cols), int(i%cols)].axis('off')
    plt.show()

In [12]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from util.util import *

def img_loader(dwipath, adcpath, maskpath):
    dwiimg = normalize(np.load(dwipath))
    adcimg = normalize(np.load(adcpath))
    maskimg = normalize(np.load(maskpath))
    return z_normalize(np.stack([dwiimg, adcimg, maskimg], axis=0))


def get_data_fname_label_in_split(data_df, fold=0, label_col='bad_outcome_3m', mode='train'):
    if mode == None:
        return data_df[['name', label_col]].values
    #return data_df[data_df['split_811_new']==mode][['name', 'bad_outcome_3m']].values
    return data_df[data_df['fold_'+str(fold)]==mode][['name', label_col]].values

def get_dataset(data_df, data_dir, mask_dir, fold=0, label_col='bad_outcome_3m', mode='train'):
    data_fname_label_arr = get_data_fname_label_in_split(data_df, fold, label_col, mode=mode)
    dwi_path_ls = sorted(load_file_path(os.path.join(data_dir, 'dwi'), NP_EXTENSION))
    adc_path_ls = sorted(load_file_path(os.path.join(data_dir, 'adc'), NP_EXTENSION))
    np_mask_path_ls = sorted(load_file_path(mask_dir, NP_EXTENSION))
    dwi_path_dict = {os.path.splitext(os.path.basename(p))[0]:p for p in dwi_path_ls}
    adc_path_dict = {os.path.splitext(os.path.basename(p))[0]:p for p in adc_path_ls}
    np_mask_path_dict = {os.path.splitext(os.path.basename(p))[0]:p for p in np_mask_path_ls}
    return [[dwi_path_dict.get(fname), adc_path_dict.get(fname), np_mask_path_dict.get(fname), label] 
            for fname, label in data_fname_label_arr if np_mask_path_dict.get(fname) if adc_path_dict.get(fname) if dwi_path_dict.get(fname)]


class AIHUB_FuncOutcomePredDataset(torch.utils.data.Dataset):
    def __init__(self, 
                 dataset_dir,
                 mask_dir,
                 dataset_df, 
                 label_col='bad_outcome_3m',
                 feature_col=['ini_nih','hx_str','age','pre_good_mrs'],
                 img_loader=img_loader, 
                 fold = 0,
                 mode='train'
    ):
        self.dataset_dir = dataset_dir
        self.mask_dir = mask_dir
        self.dataset_df_ = pd.read_csv(dataset_df)
        self.dataset_df_.drop(self.dataset_df_[self.dataset_df_['good_outcome_3m'].isnull()].index, inplace=True)
        self.dataset_df = self.dataset_df_.astype({label_col:int})
        self.feature_col = feature_col
        self.label_col = label_col
        self.img_loader = img_loader
        self.fold = fold
        self.mode = mode
        self.dataset = get_dataset(self.dataset_df, self.dataset_dir, self.mask_dir, self.fold, self.label_col, self.mode)
        
        fname_ls = [os.path.splitext(os.path.basename(data[0]))[0] for data in self.dataset]
        self.refined_dataset_df = self.dataset_df[self.dataset_df.name.isin(fname_ls)].reset_index(drop=True)
        selected_feature = self.refined_dataset_df[self.feature_col]
        
        min_max_scaler = MinMaxScaler()
        self.fitted = min_max_scaler.fit(selected_feature)
        scaled_feature = min_max_scaler.transform(selected_feature)
        scaled_feature = pd.DataFrame(scaled_feature, columns=selected_feature.columns, index=list(selected_feature.index.values))
        
        self.table_data = z_normalize(scaled_feature.values)
    
    def __getitem__(self, index):
        dwi_path, adc_path, mask_path, label = self.dataset[index]
        
        image = img_loader(dwi_path, adc_path, mask_path)
        
        table_data = torch.Tensor(self.table_data[index])
        
        return torch.Tensor(image), table_data, label
    
    def __len__(self):
        return len(self.dataset)
    

class AIHUB_FuncOutcomePredValDataset(torch.utils.data.Dataset):
    def __init__(self, 
                 dataset_dir,
                 mask_dir,
                 dataset_df, 
                 table_scaler,
                 label_col='bad_outcome_3m',
                 feature_col=['ini_nih','hx_str','age','pre_good_mrs'],
                 img_loader=img_loader, 
                 fold = 0,
                 mode='train'
    ):
        self.dataset_dir = dataset_dir
        self.mask_dir = mask_dir
        self.dataset_df_ = pd.read_csv(dataset_df)
        self.dataset_df_.drop(self.dataset_df_[self.dataset_df_['good_outcome_3m'].isnull()].index, inplace=True)
        self.dataset_df = self.dataset_df_.astype({label_col:int})
        self.feature_col = feature_col
        self.label_col = label_col
        self.table_scaler = table_scaler
        self.img_loader = img_loader
        self.fold = fold
        self.mode = mode
        self.dataset = get_dataset(self.dataset_df, self.dataset_dir, self.mask_dir, self.fold, self.label_col, self.mode)
        
        fname_ls = [os.path.splitext(os.path.basename(data[0]))[0] for data in self.dataset]
        self.refined_dataset_df = self.dataset_df[self.dataset_df.name.isin(fname_ls)].reset_index(drop=True)
        selected_feature = self.refined_dataset_df[self.feature_col]
        
        scaled_feature = self.table_scaler.transform(selected_feature)
        scaled_feature = pd.DataFrame(scaled_feature, columns=selected_feature.columns, index=list(selected_feature.index.values))
        
        self.table_data = z_normalize(scaled_feature.values)
    
    def __getitem__(self, index):
        dwi_path, adc_path, mask_path, label = self.dataset[index]
        
        image = img_loader(dwi_path, adc_path, mask_path)
        
        table_data = torch.Tensor(self.table_data[index])
        
        return torch.Tensor(image), table_data, label
    
    def __len__(self):
        return len(self.dataset)

In [13]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from util.util import *

def img_loader(dwipath, adcpath, maskpath):
    dwiimg = normalize(np.load(dwipath))
    adcimg = normalize(np.load(adcpath))
    maskimg = normalize(np.load(maskpath))
    return z_normalize(np.stack([dwiimg, adcimg, maskimg], axis=0))


def get_data_fname_label_in_split(data_df, fold=0, label_col='bad_outcome_3m', mode='train'):
    if mode == None:
        return data_df[['name', label_col]].values
    #return data_df[data_df['split_811_new']==mode][['name', 'bad_outcome_3m']].values
    return data_df[data_df['fold_'+str(fold)]==mode][['name', label_col]].values

def get_dataset(data_df, data_dir, mask_dir, fold=0, label_col='bad_outcome_3m', mode='train'):
    data_fname_label_arr = get_data_fname_label_in_split(data_df, fold, label_col, mode=mode)
    dwi_path_ls = sorted(load_file_path(os.path.join(data_dir, 'dwi'), NP_EXTENSION))
    adc_path_ls = sorted(load_file_path(os.path.join(data_dir, 'adc'), NP_EXTENSION))
    np_mask_path_ls = sorted(load_file_path(mask_dir, NP_EXTENSION))
    dwi_path_dict = {os.path.splitext(os.path.basename(p))[0]:p for p in dwi_path_ls}
    adc_path_dict = {os.path.splitext(os.path.basename(p))[0]:p for p in adc_path_ls}
    np_mask_path_dict = {os.path.splitext(os.path.basename(p))[0]:p for p in np_mask_path_ls}
    return [[dwi_path_dict.get(fname), adc_path_dict.get(fname), np_mask_path_dict.get(fname), label] 
            for fname, label in data_fname_label_arr if np_mask_path_dict.get(fname) if adc_path_dict.get(fname) if dwi_path_dict.get(fname)]
            
class AIHUB_FuncOutcomePredDataset_External(torch.utils.data.Dataset):
    def __init__(self, 
                 dataset_dir,
                 mask_dir,
                 dataset_df, 
                 label_col='bad_outcome_3m',
                 feature_col=['ini_nih','hx_str','age','pre_good_mrs'],
                 img_loader=img_loader, 
                 fold = 0,
                 mode='train'
    ):
        self.dataset_dir = dataset_dir
        self.mask_dir = mask_dir
        self.dataset_df_ = pd.read_csv(dataset_df)
        self.dataset_df_.drop(self.dataset_df_[self.dataset_df_['good_outcome_3m'].isnull()].index, inplace=True)
        self.dataset_df = self.dataset_df_.astype({label_col:int})
        self.feature_col = feature_col
        self.label_col = label_col
        self.img_loader = img_loader
        self.fold = fold
        self.mode = mode
        self.dataset = get_dataset(self.dataset_df, self.dataset_dir, self.mask_dir, self.fold, self.label_col, self.mode)
        
        fname_ls = [os.path.splitext(os.path.basename(data[0]))[0] for data in self.dataset]
        self.refined_dataset_df = self.dataset_df[self.dataset_df.name.isin(fname_ls)].reset_index(drop=True)
        selected_feature = self.refined_dataset_df[self.feature_col]
        
        scaled_feature = pd.DataFrame(selected_feature, columns=selected_feature.columns, index=list(selected_feature.index.values))
        
        self.table_data = z_normalize(scaled_feature.values)
    
    def __getitem__(self, index):
        dwi_path, adc_path, mask_path, label = self.dataset[index]
        
        image = img_loader(dwi_path, adc_path, mask_path)
        
        table_data = torch.Tensor(self.table_data[index])
        
        return torch.Tensor(image), table_data, label
    
    def __len__(self):
        return len(self.dataset)

In [14]:
dataset_dir = '/content/datas/dicom_to_np'
mask_dir = '/content/datas/dicom_to_np/mask'
dataset_df = '/content/drive/MyDrive/study/ASTROSIGHT/dataset/df_external_final.csv'
fold = 0
label_col = 'bad_outcome_3m'
feature_col = ['ini_nih', 'age', 'END']
# feature_col = ['ini_nih', 'age', 'END', 'i_glu', 'hx_str']

# train_dataset = AIHUB_FuncOutcomePredDataset(
#     dataset_dir=dataset_dir,
#     mask_dir=mask_dir,
#     dataset_df=dataset_df,
#     fold=fold,
#     label_col=label_col,
#     feature_col=feature_col,
#     mode='train'
# )

# test_dataset = AIHUB_FuncOutcomePredValDataset(
#     dataset_dir = dataset_dir,
#     mask_dir = mask_dir,
#     dataset_df = dataset_df,
#     table_scaler = train_dataset.fitted,
#     fold = fold,
#     label_col = label_col,
#     feature_col = feature_col,
#     mode='test'
# )

external_dataset = AIHUB_FuncOutcomePredDataset_External(
    dataset_dir = dataset_dir,
    mask_dir = mask_dir,
    dataset_df = dataset_df,
    fold = fold,
    label_col = label_col,
    feature_col = feature_col,
    mode = None
)

In [15]:
# train_loader = torch.utils.data.DataLoader(train_dataset,
#                                            batch_size=16, drop_last=True, shuffle=True,
#                                            pin_memory=(torch.cuda.is_available()), num_workers=0)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False,
#                                           pin_memory=(torch.cuda.is_available()), num_workers=0)
external_loader = torch.utils.data.DataLoader(external_dataset, batch_size=256, shuffle=False,
                                          pin_memory=(torch.cuda.is_available()), num_workers=0)

In [16]:
### Check average gradcam ###
for num in tqdm(range(5)):
    print(' ')
    print(f'fold: {num}')

    avg_grad_cam = np.load('/content/only_image_external_results/fold' + str(num) + '/results/gradcam_1/avg_grad_cam.npy')
    avg_grad_cam = avg_grad_cam.transpose(0,2,1)
    avg_grad_cam = normalize(z_normalize(normalize(avg_grad_cam)))*255

    fig, ax = plt.subplots(6, 6, figsize=[16,16])
    for i in range(36):
        ax[int(i/6), int(i%6)].imshow(avg_grad_cam[i,:,:], vmin=0, vmax=255)
        ax[int(i/6), int(i%6)].axis('off')
    plt.show()
    # break

Output hidden; open in https://colab.research.google.com to view.

In [17]:
external_df = external_dataset.refined_dataset_df
good_outcome_idx = external_df[external_df.bad_outcome_3m==0].index.values
bad_outcome_idx = external_df[external_df.bad_outcome_3m==1].index.values
bad_outcome_idx

array([  2,   8,  11,  13,  18,  22,  23,  25,  28,  39,  43,  47,  50,
        56,  58,  60,  67,  68,  72,  75,  77,  78,  80,  82,  83,  84,
        87,  90,  92,  96,  98,  99, 114, 115, 122, 137, 144, 150, 155,
       160, 161, 162, 164, 167, 168, 177, 179, 193, 196, 204, 210, 216])

In [18]:
### Check bad_outcome External patients ###
for num in tqdm(bad_outcome_idx):
    print(' ')
    print(f'patient_num: {num}')

    dwi = np.load(external_dataset.dataset[num][0])
    dwi = normalize(z_normalize(normalize(dwi)))*255

    fig, ax = plt.subplots(6, 6, figsize=[16,16])
    for i in range(36):
        ax[int(i/6), int(i%6)].imshow(dwi[i], cmap='gray')
        ax[int(i/6), int(i%6)].axis('off')
    plt.show()
    # break

Output hidden; open in https://colab.research.google.com to view.

In [19]:
### Grad-Cam of External patients ###
fold = 2    ## select fold
nii_path = '/content/only_image_external_results/fold' + str(fold) + '/results/gradcam_1/features.denseblock2.denselayer12/'
for num in tqdm(bad_outcome_idx):
    print(' ')
    print(f'patient_num: {num}')
    dwi = np.load(external_dataset.dataset[num][0])
    dwi = normalize(z_normalize(normalize(dwi)))*255
    scan = nib.load(nii_path + 'attention_map_' + str(num) + '_0_0.nii.gz')
    scan = scan.get_fdata()
    scan = scan.transpose(2,1,0)
    grad_cam_scan = scipy.ndimage.interpolation.zoom(scan, np.array((36,256,256))/scan.shape, mode='nearest')
    grad_cam_scan = np.transpose(grad_cam_scan, (0,2,1))
    grad_cam_scan = normalize(z_normalize(normalize(grad_cam_scan)))*255

    fig, ax = plt.subplots(6, 12, figsize=[40,20])
    for i in range(36):
        ax[int(i/6), int(i%6)+6].imshow(dwi[i], cmap='gray')
        ax[int(i/6), int(i%6)+6].axis('off')
        ax[int(i/6), int(i%6)].imshow(dwi[i], cmap='gray')
        ax[int(i/6), int(i%6)].imshow(grad_cam_scan[i], vmin=0, vmax=255, alpha=0.7)
        ax[int(i/6), int(i%6)].axis('off')
    plt.show()
    # plt.savefig('/content/cam/cam_' + str(num) + '.png')    ## save gradcam
    # plt.close(fig)
    # break

Output hidden; open in https://colab.research.google.com to view.

In [20]:
### Grad-Cam + Mask ###
fold = 2    ## select fold
nii_path = '/content/only_image_external_results/fold' + str(fold) + '/results/gradcam_1/features.denseblock2.denselayer12/'
for num in tqdm(bad_outcome_idx):
    print(' ')
    print(f'patient_num: {num}')
    dwi = np.load(external_dataset.dataset[num][0])
    mask = np.load(external_dataset.dataset[num][2])*255
    dwi = normalize(z_normalize(normalize(dwi)))*255
    scan = nib.load(nii_path + 'attention_map_' + str(num) + '_0_0.nii.gz')
    scan = scan.get_fdata()
    scan = scan.transpose(2,1,0)
    grad_cam_scan = scipy.ndimage.interpolation.zoom(scan, np.array((36,256,256))/scan.shape, mode='nearest')
    grad_cam_scan = np.transpose(grad_cam_scan, (0,2,1))
    grad_cam_scan = normalize(z_normalize(normalize(grad_cam_scan)))*255

    fig, ax = plt.subplots(6, 12, figsize=[40,20])
    for i in range(36):
        ax[int(i/6), int(i%6)+6].imshow(dwi[i], cmap='gray')
        ax[int(i/6), int(i%6)+6].imshow(mask[i], vmin=0, vmax=255, alpha=0.3)
        ax[int(i/6), int(i%6)+6].axis('off')
        ax[int(i/6), int(i%6)].imshow(dwi[i], cmap='gray')
        ax[int(i/6), int(i%6)].imshow(grad_cam_scan[i], vmin=0, vmax=255, alpha=0.7)
        ax[int(i/6), int(i%6)].axis('off')
    plt.show()
    # plt.savefig('/content/cam/cam_' + str(num) + '.png')    ## save gradcam
    # plt.close(fig)
    # break

Output hidden; open in https://colab.research.google.com to view.

In [21]:
# import glob
# from zipfile import ZipFile
# from tqdm.auto import tqdm
# import os

# with ZipFile('cam.zip', 'w') as zipper:
#     for f in tqdm(glob.glob('/content/cam/**')):
#         zipper.write(f)
#     zipper.close()